<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/paper/Paper_all_results_mcdropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/data_sets"


test_set_40k.csv    train_set_150k.csv	train_set_240k.csv
train_set_100k.csv  train_set_200k.csv	train_set_60k.csv


In [0]:
import pandas as pd
test_40k = pd.read_csv('/content/drive/My Drive/data_sets/test_set_40k.csv')
train_60k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_60k.csv')
train_100k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_100k.csv')
train_150k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_150k.csv')
train_200k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_200k.csv')
train_240k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_240k.csv')

In [0]:
full = train_240k.append(test_40k)

In [0]:
full = full[['int_rate','loan_amnt','purpose', 'profit_rate']]
full_features = full[['int_rate','loan_amnt','purpose']]
full_features = pd.get_dummies(full_features)
full_target = full[['profit_rate']]

In [0]:
def preprocess(x):
  from sklearn import preprocessing
  x = x.dropna()
 
  x_features = x[['int_rate','loan_amnt','purpose']]
  x_features = pd.get_dummies(x_features)
  x_target = x[['profit_rate']]

  scaler_features = preprocessing.MinMaxScaler().fit(full_features)
  x_features = scaler_features.transform(x_features)
  
  scaler_target = preprocessing.MinMaxScaler().fit(full_target)
  x_target = scaler_target.transform(x_target)
  
  return (x_features, x_target)

In [0]:
((test_40k_features,test_40k_target),
 (train_60k_features,train_60k_target),
 (train_100k_features,train_100k_target),
 (train_150k_features,train_150k_target),
 (train_200k_features,train_200k_target),
 (train_240k_features,train_240k_target)) = (preprocess(test_40k), 
                                             preprocess(train_60k), 
                                             preprocess(train_100k), 
                                             preprocess(train_150k), 
                                             preprocess(train_200k), 
                                             preprocess(train_240k)) 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype u

In [0]:
((test_40k_features.shape,test_40k_target.shape),
 (train_60k_features.shape,train_60k_target.shape),
 (train_100k_features.shape,train_100k_target.shape),
 (train_150k_features.shape,train_150k_target.shape),
 (train_200k_features.shape,train_200k_target.shape),
 (train_240k_features.shape,train_240k_target.shape))

(((40000, 16), (40000, 1)),
 ((65600, 16), (65600, 1)),
 ((105600, 16), (105600, 1)),
 ((155600, 16), (155600, 1)),
 ((205600, 16), (205600, 1)),
 ((245600, 16), (245600, 1)))

# Modeling


In [0]:
from keras import Model as mcModel
from keras import Input as mcInput
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.regularizers import l2

def architecture(model_type, h_cells, n_hidden, input_dim, dropout_prob, reg):
  
  
  if model_type == 'mcDropout':
    inputs = mcInput(shape=(input_dim,))
    inter = Dropout(dropout_prob)(inputs, training=True)
    inter = Dense(n_hidden[0], activation='relu',
                  W_regularizer=l2(reg))(inter)
    for i in range(len(n_hidden) - 1):
      inter = Dropout(dropout_prob)(inter, training=True)
      inter = Dense(n_hidden[i+1], activation='relu',
                    W_regularizer=l2(reg))(inter)
    inter = Dropout(dropout_prob)(inter, training=True)
    outputs = Dense(1, W_regularizer=l2(reg))(inter) 
    model3 = mcModel(inputs, outputs)
    return model3


def model_runner(X_train, y_train, X_test=test_40k_features, y_test=test_40k_target,
                dropout_prob=0.20, n_epochs=100, tau=1.0, batch_size=500, 
                lengthscale=1e-2, n_hidden=[100,100], h_cells=100):
  
  input_dim = X_train.shape[1]
  N = X_train.shape[0]
  reg = lengthscale**2 * (1 - dropout_prob) / (2. * N * tau)

  
  print('McDropout NN fit')

  model_mc_dropout = architecture(model_type = 'mcDropout',h_cells= h_cells, 
                                  n_hidden=n_hidden, input_dim=input_dim, 
                                  dropout_prob=dropout_prob, reg=reg)
  model_mc_dropout.compile(optimizer='sgd', loss='mse', metrics=['mae'])
  model_mc_dropout.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)
 
  
  return model_mc_dropout

def predictor(model_mc_dropout, 
              X_test=test_40k_features, y_test = test_40k_target, T = 1000):
  from sklearn.metrics import mean_squared_error
  import numpy as np
  
  
  probs_mc_dropout = []
  for _ in range(T):
    probs_mc_dropout += [model_mc_dropout.predict(X_test,verbose=1)]
  predictive_mean = np.mean(probs_mc_dropout, axis=0)
  predictive_variance = np.var(probs_mc_dropout, axis=0)
  mse_mc_dropout = mean_squared_error(predictive_mean, y_test)
  print(mse_mc_dropout)
  
  return probs_mc_dropout

Using TensorFlow backend.


In [0]:
model_mc_dropout_60k = model_runner(train_60k_features, train_60k_target)
probs_mc_dropout_60k = predictor(model_mc_dropout_60k)

McDropout NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
65600/65600 [==============================] - 1s 19us/step - loss: 0.0964 - mean_absolute_error: 0.2515
Epoch 2/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0571 - mean_absolute_error: 0.1927
Epoch 3/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0550 - mean_absolute_error: 0.1874
Epoch 4/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0537 - mean_absolute_error: 0.1841
Epoch 5/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0527 - mean_absolute_error: 0.1820
Epoch 6/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0520 - mean_absolute_error: 0.1803
Epoch 7/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0517 - mean_absolute_error: 0.1799
Epoch 8/100
65600/65600 [==============================] - 1s 8us/step - loss: 0.0514 - mean_absolute_error: 0.1787
Epoch 9/100
65600/65600 [==============================] - 1s 8us/step 

In [0]:

import pickle
outfile = open('probs_mc_dropout_60k.pickle','wb')
pickle.dump(probs_mc_dropout_60k,outfile)
outfile.close()


In [0]:

from google.colab import files
files.download('probs_mc_dropout_60k.pickle')

----------------------------------------
Exception happened during processing of request from ('::ffff:172.28.0.1', 43080, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfi

In [0]:
model_mc_dropout_100k = model_runner(train_100k_features, train_100k_target)
probs_mc_dropout_100k = predictor( model_mc_dropout_100k)

McDropout NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
105600/105600 [==============================] - 1s 10us/step - loss: 0.0698 - mean_absolute_error: 0.2152
Epoch 2/100
105600/105600 [==============================] - 1s 8us/step - loss: 0.0548 - mean_absolute_error: 0.1869
Epoch 3/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0532 - mean_absolute_error: 0.1833
Epoch 4/100
105600/105600 [==============================] - 1s 8us/step - loss: 0.0523 - mean_absolute_error: 0.1813
Epoch 5/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0517 - mean_absolute_error: 0.1798
Epoch 6/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0511 - mean_absolute_error: 0.1783
Epoch 7/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0508 - mean_absolute_error: 0.1780
Epoch 8/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0508 - mean_absolute_error: 0.1775
Epoch 9/100
105600/105600 [============================

In [0]:
import pickle
outfile = open('probs_mc_dropout_100k.pickle','wb')
pickle.dump(probs_mc_dropout_100k,outfile)
outfile.close()





In [0]:
from google.colab import files

files.download('probs_mc_dropout_100k.pickle')

----------------------------------------
Exception happened during processing of request from ('::ffff:172.28.0.1', 59336, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfi

In [0]:
from google.colab import files

files.download('probs_mc_dropout_100k.pickle')

NameError: ignored

In [0]:
model_mc_dropout_150k = model_runner(train_150k_features, train_150k_target)
probs_mc_dropout_150k = predictor(model_mc_dropout_150k)

McDropout NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0664 - mean_absolute_error: 0.2086
Epoch 2/100
155600/155600 [==============================] - 1s 8us/step - loss: 0.0530 - mean_absolute_error: 0.1834
Epoch 3/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0516 - mean_absolute_error: 0.1801
Epoch 4/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0510 - mean_absolute_error: 0.1783
Epoch 5/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0506 - mean_absolute_error: 0.1776
Epoch 6/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0503 - mean_absolute_error: 0.1768
Epoch 7/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0500 - mean_absolute_error: 0.1764
Epoch 8/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0498 - mean_absolute_error: 0.1758
Epoch 9/100
155600/155600 [=============================

In [0]:
import pickle
outfile = open('probs_mc_dropout_150k.pickle','wb')
pickle.dump(probs_mc_dropout_150k,outfile)
outfile.close()



In [0]:
from google.colab import files

files.download('probs_mc_dropout_150k.pickle')

----------------------------------------
Exception happened during processing of request from ('::ffff:172.28.0.1', 35914, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfi

In [0]:
model_mc_dropout_200k = model_runner(train_200k_features, train_200k_target)
probs_mc_dropout_200k = predictor( model_mc_dropout_200k)

McDropout NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
205600/205600 [==============================] - 2s 10us/step - loss: 0.0628 - mean_absolute_error: 0.2022
Epoch 2/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0537 - mean_absolute_error: 0.1840
Epoch 3/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0522 - mean_absolute_error: 0.1803
Epoch 4/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0513 - mean_absolute_error: 0.1784
Epoch 5/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0508 - mean_absolute_error: 0.1775
Epoch 6/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0504 - mean_absolute_error: 0.1767
Epoch 7/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0501 - mean_absolute_error: 0.1762
Epoch 8/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0499 - mean_absolute_error: 0.1759
Epoch 9/100
205600/205600 [============================

In [0]:

import pickle
outfile = open('probs_mc_dropout_200k.pickle','wb')
pickle.dump(probs_mc_dropout_200k,outfile)
outfile.close()



In [0]:
from google.colab import files

files.download('probs_mc_dropout_200k.pickle')

----------------------------------------
Exception happened during processing of request from ('::ffff:172.28.0.1', 51886, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfi

In [0]:
model_mc_dropout_240k = model_runner(train_240k_features, train_240k_target)
probs_mc_dropout_240k = predictor(model_mc_dropout_240k)

McDropout NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
245600/245600 [==============================] - 2s 10us/step - loss: 0.0610 - mean_absolute_error: 0.1976
Epoch 2/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0526 - mean_absolute_error: 0.1813
Epoch 3/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0516 - mean_absolute_error: 0.1791
Epoch 4/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0509 - mean_absolute_error: 0.1777
Epoch 5/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0503 - mean_absolute_error: 0.1767
Epoch 6/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0500 - mean_absolute_error: 0.1759
Epoch 7/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0498 - mean_absolute_error: 0.1757
Epoch 8/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0495 - mean_absolute_error: 0.1753
Epoch 9/100
245600/245600 [============================

In [0]:

import pickle
outfile = open('probs_mc_dropout_240k.pickle','wb')
pickle.dump(probs_mc_dropout_240k,outfile)
outfile.close()



In [0]:
from google.colab import files

files.download('probs_mc_dropout_240k.pickle')

----------------------------------------
Exception happened during processing of request from ('::ffff:172.28.0.1', 59386, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfi

In [0]:
import pickle
outfile = open('/content/drive/My Drive/probs_mc_dropout_240k.pickle','wb')
pickle.dump(probs_mc_dropout_240k,outfile)
outfile.close()

